<a href="https://colab.research.google.com/github/Benyormin/RAG-QA-bot/blob/main/RAG_QA_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Requirements

In [1]:
%%writefile requirements.txt
faiss-cpu
streamlit
pyngrok
python-dotenv
InstructorEmbedding
sentence-transformers
langchain-google-genai
google-generativeai
langchain
langchain_community


Writing requirements.txt


In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45

#Setup your API keys

You should enter your API keys below

In [4]:
%%writefile .env
GOOGLE_API_KEY= "Your_GOOGLE_API_KEY"
NGROK_TOKEN = "Your_NGROK_TOKEN"

Writing .env


# Main code

In [9]:
%%writefile helper.py
import streamlit as st
from dotenv import load_dotenv
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


#Load google API key from the .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize the GoogleGenerativeAI model
llm_gemini = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY, temperature= 0)

#Initialize our embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

def create_vector_db():
  '''
  a function that loads a CSV file and creates the vector database
  '''
  loader = CSVLoader(file_path='/content/codebasics_faqs.csv', source_column="prompt", encoding='latin-1')
  data = loader.load()
  vectordb = FAISS.from_documents(documents = data, embedding= embeddings)
  vectordb.save_local("faiss_index")

def get_qa_chain():
  '''
  this function returns a question answer chain with a stuff method
  '''
  vectordb = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization = True)
  retriever = vectordb.as_retriever()
  prompt_template = """Use the following pieces of context to answer the question at the end. If you can't find the answer, just say that you don't know ("I don't know"), don't try to make up an answer

  CONTEXT: {context}

  Question: {question} """
  PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )
  qa_chain = RetrievalQA.from_chain_type(llm = llm_gemini,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        return_source_documents=True,
                                        chain_type_kwargs={"prompt": PROMPT})
  return qa_chain

create_vector_db()

Writing helper.py


# User Interface

In [13]:
%%writefile app.py
import streamlit as st
from helper import get_qa_chain

st.title("Codebasics QA")

st.markdown("""
Welcome!
You can ask questions related to our **courses**, **website**, or **frequently asked topics**.
Our assistant will try to provide accurate answers based on our internal FAQ data.
""")

question = st.text_input("Question: ")

if question:
    chain = get_qa_chain()
    response = chain(question)
    st.header("Answer: ")
    st.write(response["result"])


Overwriting app.py


# Setup ngrok server

In [8]:
from pyngrok import ngrok
from pyngrok import ngrok, conf
from dotenv import load_dotenv
import os


# Kill existing tunnels (if any)
ngrok.kill()

# Set your authtoken
load_dotenv()

conf.get_default().auth_token = os.getenv("NGROK_TOKEN")

# Start tunnel explicitly with protocol
public_url = ngrok.connect(8501, "http")
print("Streamlit app link:", public_url)




Streamlit app link: NgrokTunnel: "https://eb1d-104-196-101-220.ngrok-free.app" -> "http://localhost:8501"


In [12]:
!streamlit run app.py &>/content/logs.txt &
